In [1]:
import pickle
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [2]:
nama_model = "klasifikasi_penyakit_jantung_xgboost.pkl"

with open(nama_model, 'rb') as file:
    model_xgb = pickle.load(file)

In [3]:
data_pasien_baru = {
    'age': [52, 65, 41],
    'sex': ['Male', 'Female', 'Male'],
    'cp': ['asymptomatic', 'typical angina', 'non-anginal'],
    'trestbps': [128, 150, 110],
    'chol': [204, 295, 235],
    'fbs': ["True", "False", "False"],
    'restecg': ['lv hypertrophy', 'normal', 'lv hypertrophy'],
    'thalch': [156, 120, 180],
    'exang': ["True", "False", "False"],
    'oldpeak': [1.0, 1.8, 0.0],
    'slope': ['flat', 'downsloping', 'upsloping'],
    'ca': [1, 2, 0],
    'thal': ['reversable defect', 'fixed defect', 'normal']
}

df_baru = pd.DataFrame(data_pasien_baru)

if model_xgb:
    # A. Mengubah nilai boolean menjadi integer jika ada
    for col in ['fbs', 'exang']:
        if df_baru[col].dtype == 'bool':
            df_baru[col] = df_baru[col].astype(int)

    # B. One-Hot Encoding untuk kolom kategori
    # Ini adalah langkah paling penting untuk menyamakan struktur fitur
    df_baru_encoded = pd.get_dummies(df_baru, dtype=int)

    # C. Menyamakan Kolom dengan Data Training
    # Model Anda dilatih dengan set kolom tertentu. Data baru harus memiliki
    # kolom yang SAMA PERSIS dan dalam URUTAN YANG SAMA.
    # Kita ambil daftar kolom dari model yang sudah dilatih.
    kolom_model = model_xgb.get_booster().feature_names
    
    # Reindex data baru agar kolomnya sama persis dengan yang diharapkan model.
    # Kolom yang tidak ada di data baru akan ditambahkan dan diisi 0.
    df_baru_final = df_baru_encoded.reindex(columns=kolom_model, fill_value=0)

    print("--- Data Pasien Baru (Setelah Preprocessing) ---")
    print("Struktur data ini sudah sesuai dengan yang diharapkan model:")
    print(df_baru_final)
    print("\n")

    # 4. MEMBUAT PREDIKSI
    prediksi = model_xgb.predict(df_baru_final)
    prediksi_proba = model_xgb.predict_proba(df_baru_final)

    # 5. MENAMPILKAN HASIL
    print("--- Hasil Prediksi untuk Pasien Baru ---")
    # Asumsi: 0 = Tidak Sakit Jantung, 1 = Sakit Jantung
    # Ini didasarkan pada cara umum dataset ini diproses.
    label_prediksi = ['Tidak Sakit Jantung' if p == 0 else 'Sakit Jantung' for p in prediksi]

    for i in range(len(df_baru)):
        print(f"--- Pasien {i+1} ---")
        print(f"Data Asli: {data_pasien_baru['sex'][i]}, {data_pasien_baru['age'][i]} tahun, cp: {data_pasien_baru['cp'][i]}")
        print(f"Prediksi: {label_prediksi[i]}")
        print(f"Probabilitas Tidak Sakit Jantung: {prediksi_proba[i][0]:.2f}")
        print(f"Probabilitas Sakit Jantung:     {prediksi_proba[i][1]:.2f}")
        print("-" * 25)


--- Data Pasien Baru (Setelah Preprocessing) ---
Struktur data ini sudah sesuai dengan yang diharapkan model:
   age  trestbps  chol  thalch  oldpeak  ca  sex_Female  sex_Male  \
0   52       128   204     156      1.0   1           0         1   
1   65       150   295     120      1.8   2           1         0   
2   41       110   235     180      0.0   0           0         1   

   cp_asymptomatic  cp_atypical angina  ...  restecg_normal  \
0                1                   0  ...               0   
1                0                   0  ...               1   
2                0                   0  ...               0   

   restecg_st-t abnormality  exang_False  exang_True  slope_downsloping  \
0                         0            0           1                  0   
1                         0            1           0                  1   
2                         0            1           0                  0   

   slope_flat  slope_upsloping  thal_fixed defect  thal_nor